<a href="https://colab.research.google.com/github/nguyenanhtienabcd/AIO2024_EXERCISE/blob/feature%2FMODULE3-WEEK5/M03W05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ex1: XGBoost Regressor

In [2]:
import numpy as np
import pandas as pd


In [3]:
lamda = 0
depth = 1
lr = 0.3

In [4]:
data = np.array([[23,24,26,27],[50, 70, 80, 85]]).T
data

array([[23, 50],
       [24, 70],
       [26, 80],
       [27, 85]])

In [5]:
# Step 01: tìm giá trị trung bình cho output
df = pd.DataFrame(data, columns=['X', 'Y'])
f0_mean = df['Y'].mean()
f0_mean

71.25

In [6]:
# Step 02: tính toán sai lệch (độ dư thừa) so với giá trị dự đoán
df['residual'] = df['Y'] - f0_mean
print(df)
sc_root = (df['residual'].sum()**2)/(len(df['residual']) + lamda)
print('similar_score_root:',sc_root)

    X   Y  residual
0  23  50    -21.25
1  24  70     -1.25
2  26  80      8.75
3  27  85     13.75
similar_score_root: 0.0


In [7]:
df_1 = df['residual']
df_1

,residual
0,-21.25
1,-1.25
2,8.75
3,13.75


In [8]:
def sc(data, lamda):
  return (data.sum()**2)/(len(data) + lamda)

In [9]:
def sc_le_ri(threshold):
  sc_left = sc(df[df['X'] < threshold]['residual'], lamda)
  sc_right = sc(df[df['X'] >= threshold]['residual'], lamda)
  return sc_left, sc_right

In [10]:
#step 3:
## 23.5
sc_l_235, sc_r_235 = sc_le_ri(23.5)
## 25
sc_l_25, sc_r_25 = sc_le_ri(25)
## 26.5
sc_l_265, sc_r_265 = sc_le_ri(26.5)

In [11]:
print(sc_l_235, sc_r_235)
print(sc_l_25, sc_r_25)
print(sc_l_265, sc_r_265)


451.5625 150.52083333333334
253.125 253.125
63.020833333333336 189.0625


In [34]:
# Step 4:
gain_235 = (sc_l_235 + sc_r_235) - sc_root
gain_25 = (sc_l_25 + sc_r_25) - sc_root
gain_265 = (sc_l_265 + sc_r_265) - sc_root
dic = {'threshold': [23.5, 25, 26.5], 'gain': [gain_235, gain_25, gain_265]}
df_gain = pd.DataFrame(dic)
df_gain


,threshold,gain
0,23.5,602.083333
1,25.0,506.250000
2,26.5,252.083333


In [43]:
threshod_max = df_gain[df_gain['gain'] == df_gain['gain'].max()]['threshold']
print(threshod_max)

0    23.5
Name: threshold, dtype: float64


In [45]:
def cal_output(threshold):
  output_l = df[df['X'] < threshold]['residual'].mean()
  output_r = df[df['X'] >= threshold]['residual'].mean()
  return output_l, output_r

In [47]:
# Step 5:
output_left = cal_output(23.5)[0]
output_right = cal_output(23.5)[1]
print(output_left, output_right)

-21.25 7.083333333333333


In [50]:
x_input = float(input('nhập giá trị X mong muốn:'))
if x_input < threshod_max.values[0]:
  print('predicted_value:',f0_mean + lr*output_left)
else:
  print('predicted_value:',f0_mean + lr*output_right)

nhập giá trị X mong muốn:25
predicted_value: 73.375


## Ex2: XGBoost Classifier

In [13]:
lamda = 0
depth = 1
lr = 0.3

In [61]:
data = np.array([[23,24,26,27],[0, 0, 1, 1]]).T
df = pd.DataFrame(data, columns=['X', 'Y'])
f0_pro = 0.5
df['residual'] = df['Y'] - f0_pro
print(df)

    X  Y  residual
0  23  0      -0.5
1  24  0      -0.5
2  26  1       0.5
3  27  1       0.5


In [62]:
# Step 2:
def sc(data, lamda):
  sum_pro = f0_pro * (1-f0_pro)
  for i in range(len(data)-1):
    sum_pro += f0_pro * (1-f0_pro)
  return (data.sum()**2)/(sum_pro + lamda)

In [63]:
def sc_le_ri(threshold):
  sc_left = sc(df[df['X'] < threshold]['residual'], lamda)
  sc_right = sc(df[df['X'] >= threshold]['residual'], lamda)
  return sc_left, sc_right

In [65]:
sc_root = sc(df['residual'], lamda)
print(sc_root)

0.0


In [66]:
# Step 3:
def gain(threshold, sc_root):
  sc_l, sc_r = sc_le_ri(threshold)
  return (sc_l + sc_r) - sc_root
df_gain = pd.DataFrame({'threshold': [23.5, 25, 26.5], 'gain': [gain(23.5, sc_root), gain(25, sc_root), gain(26.5, sc_root)]})
df_gain

,threshold,gain
0,23.5,1.333333
1,25.0,4.000000
2,26.5,1.333333


In [70]:
# Step 4:
threshod_max = df_gain[df_gain['gain'] == df_gain['gain'].max()]['threshold'].values[0]
print(threshod_max)

25.0


In [68]:
def cal_output(threshold):
  output_l = df[df['X'] < threshold]['residual'].sum()/len(df[df['X'] < threshold]['residual'])
  output_r = df[df['X'] >= threshold]['residual'].sum()/len(df[df['X'] >= threshold]['residual'])
  return output_l, output_r

In [71]:
output_left = cal_output(threshod_max)[0]
output_right = cal_output(threshod_max)[1]
print(output_left, output_right)

-0.5 0.5


In [73]:
def cal_pro(f0, output):
  log_pred = np.log(f0/(1-f0)) + lr*output
  return np.exp(log_pred)/(1+np.exp(log_pred))

In [76]:
x_input = float(input('nhập giá trị X mong muốn:'))
if x_input < threshod_max:
  print('predicted_value:',round(cal_pro(f0_pro, output_left),0))
else:
  print('predicted_value:',round(cal_pro(f0_pro, output_right),0))

nhập giá trị X mong muốn:25
predicted_value: 1.0


## Ex3:

In [17]:
import numpy as np
import pandas as pd
import matplotlib . pyplot as plt
import xgboost as xgb
from sklearn . metrics import mean_squared_error , mean_absolute_error
from sklearn . preprocessing import OrdinalEncoder
from sklearn . model_selection import train_test_split

In [18]:
!gdown 1xwJmYJxEia06sxUdJyGO7JFx4DNK1fbp

Downloading...
From: https://drive.google.com/uc?id=1xwJmYJxEia06sxUdJyGO7JFx4DNK1fbp
To: /content/Problem3.csv
100% 37.4k/37.4k [00:00<00:00, 35.2MB/s]


In [19]:
dataset_path = "/content/Problem3.csv"
data_df = pd.read_csv( dataset_path )

In [20]:
# chuyển đổi sang list và đếm số lượng các phần tử unique trong dữ liệu
categorical_cols = data_df.select_dtypes(include =['object', 'bool']).columns.to_list()
for col_name in categorical_cols :
  n_categories = data_df[col_name].nunique()
  print (f'Number of categories in { col_name }:{ n_categories }')

# chuyển đổi dữ liệu sạng string sang số
ordinal_encoder = OrdinalEncoder ()
encoded_categorical_cols = ordinal_encoder.fit_transform(data_df [
categorical_cols])

# tạo một df chứa các dữ liệu số
encoded_categorical_df = pd.DataFrame(
encoded_categorical_cols,
columns = categorical_cols
)

# loại bỏ hàng có chứa các thư mục
numerical_df = data_df.drop( categorical_cols, axis =1)
encoded_df = pd. concat([ numerical_df , encoded_categorical_df ], axis =1)

Number of categories in month: 12
Number of categories in day: 7
Number of categories in rain: 2


In [21]:
# tạo dữ liệu đầu vào và đầu ra
X = encoded_df.drop( columns =['area'])
y = encoded_df ['area']

In [22]:
# tách dữ liệu thành dữ liệu train và dữ liệu test
X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size =0.3 ,
random_state =7
)

In [23]:
# training model
xg_reg = xgb.XGBRegressor(seed =7,
learning_rate =0.01,
n_estimators =102,
max_depth =3
)
xg_reg .fit( X_train , y_train )

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=102, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [24]:
# dự đoán model
preds = xg_reg.predict(X_test )

mae = mean_absolute_error(y_test, preds )
mse = mean_squared_error(y_test, preds )

print ('Evaluation results on test set:')
print (f'Mean Absolute Error: { mae }')
print (f'Mean Squared Error : { mse}')

Evaluation results on test set:
Mean Absolute Error: 1.1484401341167767
Mean Squared Error : 1.8845074196256495


## Ex4:

In [25]:
import numpy as np
import pandas as pd
import matplotlib . pyplot as plt
import xgboost as xgb
from sklearn . metrics import accuracy_score
from sklearn . preprocessing import LabelEncoder
from sklearn . model_selection import train_test_split

In [26]:
!gdown 1pVdH-2b_odeuEPdXbLQYDcHXxgqqBK4i

Downloading...
From: https://drive.google.com/uc?id=1pVdH-2b_odeuEPdXbLQYDcHXxgqqBK4i
To: /content/Problem4.csv
100% 12.3k/12.3k [00:00<00:00, 24.9MB/s]


In [27]:
dataset_path = "/content/Problem4.csv"
data_df = pd. read_csv ( dataset_path )

In [28]:
X, y = data_df . iloc [: ,: -1] , data_df . iloc [: , -1]

In [29]:
X_train , X_test , y_train , y_test = train_test_split (X, y,
test_size =0.3 ,
random_state =7
)

In [30]:
xg_class = xgb. XGBClassifier ( seed =7)
xg_class .fit ( X_train , y_train )

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [31]:
preds = xg_class . predict ( X_test )
preds

array([2, 0, 2, 2, 1, 2, 1, 0, 1, 2, 0, 1, 2, 1, 1, 1, 1, 2, 0, 0, 1, 1,
       1, 1, 0, 2, 1, 2, 1, 2, 1, 0, 2, 1, 1, 1, 2, 2, 0, 2, 0, 1, 2, 2,
       1, 0, 0, 1, 1, 0, 1, 1, 0, 0])

In [32]:
train_acc = accuracy_score ( y_train , xg_class . predict ( X_train ))
test_acc = accuracy_score (y_test , preds )

print (f'Train ACC: { train_acc }')
print (f'Test ACC: { test_acc }')

Train ACC: 1.0
Test ACC: 0.9629629629629629
